# Update DB 
`get_additional_data` must be run prior (OR retrieve CSVs from google drive <a href="https://drive.google.com/drive/folders/1s_6b-szIrBIscnAwA6HV__kgFDCoOlpd?usp=sharing">here</a>)

This file uses CSVs generated from get_additional_data (descriptions, trailers, posters) and get_service_provider (service provider links) to update movie data in `imdb_movies` table. 

It may make sense to have `service_providers` as a seperate table and create a one to many relationship between the movie and their service providers. 

#### New movies table info:
New table columns:
`movie_id
title_type
primary_title
original_title
is_adult
start_year
end_year
runtime_minutes
genres
poster_url
average_rating
num_votes
original_language
description
trailer_url`

New table schema:
`
    movie_id character varying COLLATE pg_catalog."default" NOT NULL,
    title_type character varying COLLATE pg_catalog."default",
    primary_title character varying COLLATE pg_catalog."default",
    original_title character varying COLLATE pg_catalog."default",
    is_adult boolean,
    start_year integer,
    end_year integer,
    runtime_minutes integer,
    genres character varying COLLATE pg_catalog."default",
    poster_url character varying COLLATE pg_catalog."default",
    average_rating real,
    num_votes integer,
    original_language character varying COLLATE pg_catalog."default",
    description character varying COLLATE pg_catalog."default",
    trailer_url character varying COLLATE pg_catalog."default",
`

In [3]:
# imports 
import pandas as pd 
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv

In [4]:
# load .env file for keys
load_dotenv()

True

In [5]:
# create connection to new prod DB
connection = psycopg2.connect(
                            user=os.getenv('DB_USER'),
                            password=os.getenv('DB_PASSWORD'),
                            host=os.getenv('HOST'),
                            port=os.getenv('PORT'),
                            database=os.getenv('DB_NAME')
                            )

## Add descriptions to movies

In [57]:
# read in description data
desc_df = pd.read_csv('../../data/description_results.csv', engine='python')
print(desc_df.shape)
desc_df.head()

(201352, 3)


,movie_id,tmdb_id,description
0,1051231,31223.0,In the Hands of the Gods is the true story of ...
1,1051244,573815.0,A group of talented youth exploited by the hea...
2,10513474,639651.0,A unique chance to explore Pier Paolo Pasolini...
3,10515086,599672.0,A meeting with a new inmate in the psychiatric...
4,10515340,531678.0,A strange disease is plaguing the city. Hoping...


In [58]:
# how many null values
desc_df['description'].isnull().sum()

27476

In [59]:
# any empty strings?
desc_df[desc_df['description'] == ""].shape[0]

0

In [60]:
# drop rows without a description
desc_df = desc_df.dropna(subset=['description'])
desc_df.shape

(173876, 3)

In [61]:
# fix movie_id
desc_df['movie_id'] = desc_df['movie_id'].astype(str)
def fill_id(id):
    """Adds leading zeroes back if necessary. This makes the id match the database."""
    if len(str(id)) < 7:
        length = len(str(id))
        id = "0"*(7 - length) + str(id)
    return str(id)

desc_df['movie_id'] = desc_df['movie_id'].apply(fill_id)

In [62]:
# update every movie we have a description for
cursor = connection.cursor()
update_query = "UPDATE movies SET description = %s WHERE movie_id = %s;"

for movie_id, desc in desc_df[['movie_id', 'description']].values:
    cursor.execute(update_query, (desc, movie_id))

connection.commit()
cursor.close()

In [80]:
# count every movie with a description
cursor = connection.cursor()

query = "SELECT COUNT(*) FROM movies WHERE description IS NOT NULL;"
cursor.execute(query)
desc_count = cursor.fetchone()

cursor.close()
desc_count

(173876,)

In [66]:
# get a test_id and desc
test_id = desc_df['movie_id'][0]
test_desc = desc_df['description'][0]
test_id, test_desc

('1051231',
 'In the Hands of the Gods is the true story of five young British freestyle footballers journey across the Americas to Argentina in the hope of meeting their hero, Diego Maradona. This coming-of-age road movie tells the story of a group of young men in pursuit of a lifelong dream.')

In [70]:
# ensure test_desc is same in DB
cursor = connection.cursor()
query = f"SELECT movie_id, description FROM movies WHERE movie_id = '{test_id}';"
cursor.execute(query)
print(cursor.fetchone())
cursor.close()

('1051231', 'In the Hands of the Gods is the true story of five young British freestyle footballers journey across the Americas to Argentina in the hope of meeting their hero, Diego Maradona. This coming-of-age road movie tells the story of a group of young men in pursuit of a lifelong dream.')


## Add trailer to movie

In [71]:
# read in trailer data 
trailer_df = pd.read_csv('../../data/trailer_data.csv')
print(trailer_df.shape)
trailer_df.head()

(50793, 4)


,movie_id,video_key,video_site,more_than_one
0,1051244,ztSS7hnEviY,YouTube,False
1,10515086,WA2NvFSHchk,YouTube,False
2,10515460,HQksgesFrFY,YouTube,False
3,10515480,QBNKpcUOWgI,YouTube,False
4,1051232,k9SdzYiyG14,YouTube,False


In [72]:
# any null values?
trailer_df.isnull().sum()

movie_id         0
video_key        0
video_site       0
more_than_one    0
dtype: int64

In [73]:
# what are the video_site values?
trailer_df['video_site'].value_counts()

YouTube    50205
Vimeo        588
Name: video_site, dtype: int64

In [74]:
# how many did have more than one on TMDb
trailer_df['more_than_one'].value_counts()

False    43481
True      7312
Name: more_than_one, dtype: int64

In [75]:
# fix movie_id
trailer_df['movie_id'] = trailer_df['movie_id'].astype(str)
trailer_df['movie_id'] = trailer_df['movie_id'].apply(fill_id)

In [81]:
# update every movie we have a trailer for
cursor = connection.cursor()
youtube_base = 'https://www.youtube.com/watch?v='
vimeo_base = 'https://vimeo.com/'

for movie_id, key, site in trailer_df[['movie_id', 'video_key', 'video_site']].values:
    if site == 'YouTube':
        trailer_url = youtube_base + str(key)
    else:
        trailer_url = vimeo_base + str(key)
    update_query = f"UPDATE movies SET trailer_url = '{trailer_url}' WHERE movie_id = '{movie_id}';"
    cursor.execute(update_query)

connection.commit()
cursor.close()

In [82]:
# count every movie with a trailer_url
cursor = connection.cursor()

query = "SELECT COUNT(*) from movies WHERE trailer_url IS NOT NULL;"
cursor.execute(query)
trailer_count = cursor.fetchone()

cursor.close()
trailer_count

(50793,)

In [89]:
# get test_id and key
test_id = trailer_df['movie_id'][1000]
test_key = trailer_df['video_key'][1000]
test_id, test_key

('1209379', 'AHne0pCugmw')

In [90]:
# ensure test_key is same in DB
cursor = connection.cursor()
query = f"SELECT movie_id, trailer_url FROM movies WHERE movie_id = '{test_id}';"
cursor.execute(query)
print(cursor.fetchone())
cursor.close()

('1209379', 'https://www.youtube.com/watch?v=AHne0pCugmw')


## Add poster to movies that didn't have one

In [11]:
# read in poster data 
poster_df = pd.read_csv('../../data/poster_data.csv')
print(poster_df.shape)
poster_df.head()

(201482, 3)


,movie_id,tmdb_id,poster_path
0,1051259,87397,/4c27iElQGoLSlrg2uZQUANHA4aa.jpg
1,1051834,533781,NaN
2,10520386,637881,/iASQXlZqZfERdwTnBxIRpfgKrsO.jpg
3,1052347,201899,NaN
4,10519798,676343,NaN


In [12]:
# how many null poster_paths?
poster_df['poster_path'].isnull().sum()

77522

In [13]:
# drop those rows
poster_df = poster_df.dropna(subset=['poster_path'])
poster_df.shape

(123960, 3)

In [16]:
# fix movie_id
poster_df['movie_id'] = poster_df['movie_id'].astype(str)
poster_df['movie_id'] = poster_df['movie_id'].apply(fill_id)

In [9]:
# count every movie with a poster_path before update
cursor = connection.cursor()

query = "SELECT COUNT(*) from movies WHERE poster_url IS NOT NULL;"
cursor.execute(query)
poster_count = cursor.fetchone()

cursor.close()
poster_count

(183087,)

In [17]:
# update every movie we have a poster for
cursor = connection.cursor()
base_query = "UPDATE movies SET poster_url = %s WHERE movie_id = %s;"

for movie_id, poster_path in poster_df[['movie_id', 'poster_path']].values:
    cursor.execute(base_query, (poster_path, movie_id))

connection.commit()
cursor.close()

In [21]:
# count every movie with a poster_path after update
cursor = connection.cursor()

query = "SELECT COUNT(*) from movies WHERE poster_url IS NOT NULL;"
cursor.execute(query)
poster_count = cursor.fetchone()

cursor.close()
poster_count

(194343,)

## Add providers to DB

In [1]:
from justwatch import JustWatch
# make justwatch instance
just_watch = JustWatch(country='US')

In [2]:
# make providers list to narrow search
provider_details = just_watch.get_providers()
providers = []

for provider in provider_details:
    if len(provider['data']['deeplink_data']) > 0:
        providers.append((
            provider['id'], 
            provider['clear_name']
        ))

providers[0], len(providers)

((8, 'Netflix'), 49)

In [9]:
# count every provider before adding to DB
cursor = connection.cursor()

query = "SELECT COUNT(*) from providers;"
cursor.execute(query)
provider_count = cursor.fetchone()

cursor.close()
provider_count

(0,)

In [10]:
# add all providers
query = "INSERT INTO providers (provider_id, name) VALUES (%s, %s);"
cursor = connection.cursor()

for prov in providers:
    cursor.execute(query, (prov[0], prov[1]))

connection.commit()
cursor.close()

In [4]:
# count every provider after adding to DB
cursor = connection.cursor()

query = "SELECT COUNT(*) from providers;"
cursor.execute(query)
provider_count = cursor.fetchone()

cursor.close()
provider_count

(49,)

In [12]:
# make sure ids are correct, will use netflix to test
cursor = connection.cursor()

query = "SELECT provider_id, name from providers WHERE name = 'Netflix';"
cursor.execute(query, 'Netflix')
netflix_id = cursor.fetchone()

cursor.close()
netflix_id

(8, 'Netflix')

In [7]:
# creat an our_provider_ids to check against ids in df
cursor = connection.cursor()

query = "SELECT provider_id from providers;"
cursor.execute(query)
our_provider_ids = cursor.fetchall()

cursor.close()
our_provider_ids[0]

(8,)

In [8]:
# clean up 
for idx, provider in enumerate(our_provider_ids):
    our_provider_ids[idx] = provider[0]

our_provider_ids[0]

8

## Add service provider links to movies

In [9]:
# if multiple files
# prov_df1 = pd.read_csv('provider_data_0_100k.csv')
# prov_df2 = pd.read_csv('provider_data_100k_200k.csv')
# prov_df3 = pd.read_csv('provider_data_200k_on.csv')
# provider_df = pd.concat([prov_df1, prov_df2, prov_df3])
# if one file
provider_df = pd.read_csv('provider_data.csv')
print(provider_df.shape)
provider_df.head()

(217219, 9)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,presentation_types,monetization_type,ratio
0,1051320,La cantatrice chauve,81030,The Golden Compass,"7,7,7,7,3,3,192,192,10,10,10,10,358,358,2,2,2,...",https://www.vudu.com/content/movies/details/Th...,"sd,sd,hd,hd,hd,hd,hd,hd,hd,sd,hd,sd,hd,hd,hd,s...","rent,buy,buy,rent,rent,buy,buy,rent,rent,rent,...",26
1,10513286,Historia de mi nombre,403709,Marriage Story,"8,8,8","http://www.netflix.com/title/80223779,http://w...","hd,sd,4k","flatrate,flatrate,flatrate",40
2,1051403,Whiskey and Water,78787,Water Lilies,"9,9,258,258",https://www.amazon.com/gp/product/B004POVM2W?c...,"hd,sd,hd,sd","flatrate,flatrate,flatrate,flatrate",34
3,10514608,"Daniel, Debra, Leslie (and You?)",684228,"Geek, and You Shall Find","2,2,2,2,7,7,7,7,10,10,10,10,3,192,3,192,3,192,...",https://itunes.apple.com/us/movie/geek-and-you...,"sd,hd,sd,hd,sd,hd,sd,hd,hd,sd,hd,sd,hd,hd,sd,s...","buy,rent,rent,buy,buy,rent,rent,buy,buy,buy,re...",39
4,10514932,What Do I Do Now?,538893,What the Night Can Do,"10,10,10,10,73",https://www.amazon.com/gp/product/B083MSWV48?c...,"sd,hd,sd,hd,sd","buy,rent,rent,buy,ads",53


We need to filter out rows with `ratio` values less than 90 because it is very likely it is not the correct movie if less than that.

In [10]:
filtered_df = provider_df[provider_df['ratio'] > 90]
print(filtered_df.shape)
filtered_df.head()

(40589, 9)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,presentation_types,monetization_type,ratio
67,10532840,Who Killed Garrett Phillips?,818172,Who Killed Garrett Phillips?,"31,31,27,27,358,68,68,10,10,7,7,192,192,3,3,2,2",http://play.hbogo.com/feature/urn:hbo:feature:...,"hd,sd,hd,sd,hd,hd,sd,hd,sd,hd,sd,hd,sd,hd,sd,h...","flatrate,flatrate,flatrate,flatrate,flatrate,b...",100
84,1053818,Butch Jamie,252612,Butch Jamie,"10,10,10,10,191",https://www.amazon.com/gp/product/B01N8X0B2Q?c...,"hd,sd,hd,sd,sd","buy,buy,rent,rent,flatrate",100
205,1056433,My Monster Mom,221135,My Monster Mom,"9,9,10,10",https://www.amazon.com/gp/product/B076C6N4QZ?c...,"hd,sd,sd,sd","flatrate,flatrate,rent,buy",100
240,1056441,Tru Loved,28075,Tru Loved,"10,10",https://www.amazon.com/gp/product/B07KXVBDZ9?c...,"sd,sd","buy,rent",100
247,10579944,Brexit Behind Closed Doors,811212,Brexit: Behind Closed Doors,"10,10,10,10,2,2,2,2,192,192,192,3,3,3,9,9",https://www.amazon.com/gp/product/B07TCYLB6F?c...,"hd,sd,hd,sd,hd,sd,hd,sd,hd,hd,sd,hd,hd,sd,sd,hd","buy,buy,rent,rent,buy,buy,rent,rent,rent,buy,b...",98


In [11]:
def fill_id(id):
    """Adds leading zeroes back if necessary. This makes the id match the database."""
    if len(str(id)) < 7:
        length = len(str(id))
        id = "0"*(7 - length) + str(id)
    return str(id)

In [12]:
# fix movie_id
prov_df = filtered_df.copy()
prov_df['movie_id'] = prov_df['movie_id'].astype(str)
prov_df['movie_id'] = prov_df['movie_id'].apply(fill_id)

In [13]:
# check new df
prov_df.head()

,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,presentation_types,monetization_type,ratio
67,10532840,Who Killed Garrett Phillips?,818172,Who Killed Garrett Phillips?,"31,31,27,27,358,68,68,10,10,7,7,192,192,3,3,2,2",http://play.hbogo.com/feature/urn:hbo:feature:...,"hd,sd,hd,sd,hd,hd,sd,hd,sd,hd,sd,hd,sd,hd,sd,h...","flatrate,flatrate,flatrate,flatrate,flatrate,b...",100
84,1053818,Butch Jamie,252612,Butch Jamie,"10,10,10,10,191",https://www.amazon.com/gp/product/B01N8X0B2Q?c...,"hd,sd,hd,sd,sd","buy,buy,rent,rent,flatrate",100
205,1056433,My Monster Mom,221135,My Monster Mom,"9,9,10,10",https://www.amazon.com/gp/product/B076C6N4QZ?c...,"hd,sd,sd,sd","flatrate,flatrate,rent,buy",100
240,1056441,Tru Loved,28075,Tru Loved,"10,10",https://www.amazon.com/gp/product/B07KXVBDZ9?c...,"sd,sd","buy,rent",100
247,10579944,Brexit Behind Closed Doors,811212,Brexit: Behind Closed Doors,"10,10,10,10,2,2,2,2,192,192,192,3,3,3,9,9",https://www.amazon.com/gp/product/B07TCYLB6F?c...,"hd,sd,hd,sd,hd,sd,hd,sd,hd,hd,sd,hd,hd,sd,sd,hd","buy,buy,rent,rent,buy,buy,rent,rent,rent,buy,b...",98


In [14]:
# count every movie_provider before update
cursor = connection.cursor()

query = "SELECT COUNT(*) from movie_providers;"
cursor.execute(query)
mov_prov_count = cursor.fetchone()

cursor.close()
mov_prov_count

(0,)

In [15]:
# add all providers
cursor = connection.cursor()
query = """
INSERT INTO movie_providers 
(movie_id, provider_id, provider_movie_url, presentation_type, monetization_type)
VALUES (%s, %s, %s, %s, %s);"""
random_provider_ids = []
errors = []
counter = 0

for movie in prov_df.values:
    provider_ids = movie[4].split(',')
    urls = movie[5].split(',')
    pres_types = movie[6].split(',')
    mon_types = movie[7].split(',')
    for i in range(len(provider_ids)):
        counter += 1
        try:
            if int(provider_ids[i]) in our_provider_ids:
                cursor.execute(
                    query, 
                    (movie[0], provider_ids[i], urls[i], pres_types[i], mon_types[i]))
            else:
                random_provider_ids.append(provider_ids[i])
        except Exception as e:
            errors.append((movie[0], provider_ids[i]))
        if counter % 10000 == 0:
            print(f"Saving at: {counter}")
            print(f"Error count: {len(errors)}")
            print(f"Random prov count: {len(random_provider_ids)}")
            connection.commit()

cursor.close()

Saving at: 10000
Error count: 0
Random prov count: 2330
Saving at: 20000
Error count: 0
Random prov count: 4731
Saving at: 30000
Error count: 0
Random prov count: 6899
Saving at: 40000
Error count: 0
Random prov count: 9392
Saving at: 50000
Error count: 0
Random prov count: 12355
Saving at: 60000
Error count: 1
Random prov count: 15347
Saving at: 70000
Error count: 1
Random prov count: 18337
Saving at: 80000
Error count: 1
Random prov count: 21227
Saving at: 90000
Error count: 1
Random prov count: 24112
Saving at: 100000
Error count: 3
Random prov count: 27008
Saving at: 110000
Error count: 4
Random prov count: 29838
Saving at: 120000
Error count: 5
Random prov count: 32792
Saving at: 130000
Error count: 7
Random prov count: 35770
Saving at: 140000
Error count: 7
Random prov count: 38726
Saving at: 150000
Error count: 8
Random prov count: 41492
Saving at: 160000
Error count: 9
Random prov count: 44122
Saving at: 170000
Error count: 9
Random prov count: 46817
Saving at: 180000
Error cou

In [16]:
# count every movie_provider after update
cursor = connection.cursor()

query = "SELECT COUNT(*) from movie_providers;"
cursor.execute(query)
mov_prov_count = cursor.fetchone()

cursor.close()
mov_prov_count

(426850,)

In [17]:
# check worst ratios 
prov_df[['title', 'jw_title', 'ratio']].sort_values(by='ratio')[:20]

,title,jw_title,ratio
121519,U.F.O,U.F.O.,91
47695,To Each Their Own,"To Each, Her Own",91
117169,"Shake, Rattle & Rock!","Shake, Rattle and Rock!",91
180245,Il Generale Della Rovere,General Della Rovere,91
209898,An American Girl on the Home Front,Molly: An American Girl on the Home Front,91
91800,Stoned Bros,Stone Bros.,91
201123,Female Prisoner Scorpion: #701's Grudge Song,Female Prisoner Scorpion: Grudge Song,91
132064,Blame,Blame!,91
162318,"Shake, Rattle & Roll 9","Shake, Rattle and Roll 9",91
201460,Mirror Mirror 4: Reflections,"Mirror, Mirror IV: Reflection",91


In [18]:
prov_df.dtypes

movie_id              object
title                 object
jw_id                  int64
jw_title              object
offer_provider_id     object
offer_urls            object
presentation_types    object
monetization_type     object
ratio                  int64
dtype: object

`offer_provider_id` and `offer_urls` are string deliminated sequences. We can use the JustWatch API to get the provider details of each ID and the URLs will be saved to be linked on the movie detail page. There is other info we could've saved to better differentiate between the values with the same provider_id on the same movie. From what I saw it seems some are HD while some are SD, as well as some of the monetization types differ resulting in repeats of the same provider. 

We will probably want to filter out most service providers (other than Netflix, Amazon, and Hulu) because some are not popular enough to justify displaying theirs on the webpage. 